In [1]:
!pip install -U torch
!pip install transformers==4.28.0
!pip install -U datasets

  Using cached torch-2.0.1-cp310-cp310-win_amd64.whl (172.3 MB)
  Using cached filelock-3.12.0-py3-none-any.whl (10 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)


You should consider upgrading via the 'C:\Studia\pythonProject\venv\Scripts\python.exe -m pip install --upgrade pip' command.


  Using cached transformers-4.28.0-py3-none-any.whl (7.0 MB)
  Using cached huggingface_hub-0.14.1-py3-none-any.whl (224 kB)
  Using cached requests-2.30.0-py3-none-any.whl (62 kB)
  Using cached numpy-1.24.3-cp310-cp310-win_amd64.whl (14.8 MB)
  Using cached regex-2023.5.5-cp310-cp310-win_amd64.whl (267 kB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-win_amd64.whl (3.5 MB)
  Using cached fsspec-2023.5.0-py3-none-any.whl (160 kB)
  Using cached charset_normalizer-3.1.0-cp310-cp310-win_amd64.whl (97 kB)
  Using cached certifi-2023.5.7-py3-none-any.whl (156 kB)
  Using cached urllib3-2.0.2-py3-none-any.whl (123 kB)


You should consider upgrading via the 'C:\Studia\pythonProject\venv\Scripts\python.exe -m pip install --upgrade pip' command.


  Using cached datasets-2.12.0-py3-none-any.whl (474 kB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Using cached pandas-2.0.1-cp310-cp310-win_amd64.whl (10.7 MB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached pyarrow-12.0.0-cp310-cp310-win_amd64.whl (21.5 MB)
  Using cached xxhash-3.2.0-cp310-cp310-win_amd64.whl (30 kB)
  Using cached aiohttp-3.8.4-cp310-cp310-win_amd64.whl (319 kB)
  Using cached multiprocess-0.70.14-py310-none-any.whl (134 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached yarl-1.9.2-cp310-cp310-win_amd64.whl (61 kB)
  Using cached multidict-6.0.4-cp310-cp310-win_amd64.whl (28 kB)
  Using cached frozenlist-1.3.3-cp310-cp310-win_amd64.whl (33 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)


You should consider upgrading via the 'C:\Studia\pythonProject\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
import transformers
import pandas as pd
import numpy as np
from datasets import Dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

metric = load_metric('glue', 'sst2')

def process(x):
  return tokenizer(x['text'])

def compute_metrics(eval_preds):
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

data = pd.read_csv("tagged_data.csv", index_col=0)
data.rename(columns={"Tweets": "text", "cluster": "label"}, inplace=True)
dataset_ = Dataset.from_pandas(data)
dataset = dataset_.train_test_split(0.1)

model_checkpoint = 'distilbert-base-uncased'
batch_size = 128

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

train_ds = dataset['train'].map(process)
test_ds = dataset['test'].map(process)

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

args = TrainingArguments(
    f'{model_checkpoint}_sentiment_analysis',
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = 5,
    weight_decay = 0.01,
    load_best_model_at_end = True,
    metric_for_best_model = 'accuracy'
)

trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.evaluate([train_ds[0]])

trainer.train()

Map:   0%|          | 0/12640 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (907 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/1405 [00:00<?, ? examples/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classi

C:\Studia\pythonProject\venv\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


RuntimeError: The size of tensor a (724) must match the size of tensor b (512) at non-singleton dimension 1

In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

text = "i love this positive good thing"
inputs = tokenizer(text, return_tensors='pt')
input_ids = inputs['input_ids'].to(device)
attention_mask = inputs['attention_mask'].to(device)

with torch.no_grad():
  output = model(input_ids=input_ids, attention_mask=attention_mask)
  logits = output.logits
  predictions = torch.argmax(logits, dim=-1)

In [ ]:
if predictions.item() == 1:
  print('Positive')
else:
  print('Negative')

In [2]:
# prosty klasyfikator sentymentu z wykorzystaniem gotowego modelu BERT:
from transformers import BertTokenizer, BertForSequenceClassification, pipeline

bert = BertForSequenceClassification.from_pretrained(
        'distilbert-base-uncased', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('distilbert-base-uncased')


model = pipeline('text-classification',
               model=bert,
               tokenizer=tokenizer)

sentence1_result = model('This is a good thing, nice!')

print(sentence1_result)

ImportError: 
BertForSequenceClassification requires the PyTorch library but it was not found in your environment.
However, we were able to find a TensorFlow installation. TensorFlow classes begin
with "TF", but are otherwise identically named to our PyTorch classes. This
means that the TF equivalent of the class you tried to import would be "TFBertForSequenceClassification".
If you want to use TensorFlow, please use TF classes instead!

If you really do want to use PyTorch please go to
https://pytorch.org/get-started/locally/ and follow the instructions that
match your environment.
